In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('./HDF5Module')
sys.path.append('./KMeans')
import readh5
from sklearn.cluster import KMeans
import scipy.ndimage
import scipy.misc
rotate = scipy.ndimage.interpolation.rotate
resize = scipy.misc.imresize


In [5]:
data = readh5.mergeDataSet('../weiner', r'(.*?)weiner\.hdf')
data2 = np.zeros((data.shape[0] * 8,) + (data.shape[1] / 3,) + (data.shape[2] / 3,))
for i in range(data.shape[0]):
    for j in range(8):
        datai = data[i]
        data2[8 * i + j] = resize(rotate(datai ,45 * j, reshape=False), (data.shape[1] / 3,) + (data.shape[2] / 3,))
with h5py.File('rotated.hdf','w') as h:
    h.create_group('images')
    for i in range(data.shape[0]):
        for j in range(8):
            h['images'].create_dataset(
                str(i) + '_' + str(j*45),
                shape=data2[8 * i + j, :].shape,
                dtype='float32',
                data=data2[8 * i + j, :])

            

Successfully load data


In [ ]:
data = data2

In [13]:
kclf = KMeans(n_clusters=100, random_state=0).fit(data)
pred = kclf.predict(data)
print pred, len(pred)


[27 62  4 ...,  0  0  0] 9832


In [14]:
print type(pred)

<type 'numpy.ndarray'>


In [15]:
stacked = np.zeros((100,) + (data.shape[1],))


In [16]:

x = np.where(pred == 2)
dt = [data[j] for j in x]
print dt[0].shape
a = np.sum(dt[0],0)/float(dt[0].shape[0])
print a.shape

(1, 32400)
(32400,)


In [17]:
for i in range(100):
    if i in pred:
        index = np.where(pred == i)
        dt = [data[j] for j in index]
        stacked[i] = np.sum(dt[0],0)/float(dt[0].shape[0])

In [18]:
print stacked.shape

(100, 32400)


In [19]:
result = h5py.File('clusterResult_masked.hdf', 'w')
result.create_group('MDF')
result['MDF'].create_group('images')
for i in range(stacked.shape[0]):
    images = result['MDF']['images']
    images.create_group(str(i))
    images[str(i)].create_dataset(
        'image',
        shape=(180, 180),
        dtype='float32',
        data=kclf.cluster_centers_[i, :].reshape(180, 180))
result.close()
print kclf.cluster_centers_.shape

(100, 32400)
